In [3]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [4]:
# Load the dataset 
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(["RowNumber", 'CustomerId', 'Surname'], axis = 1)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

ohe_geo = OneHotEncoder()
geo_encoder = ohe_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

# Combine OHE columns with the original data
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
# Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

In [5]:
# Divide into independent and dependent features 
# Predict the output of Exited based on other column inputs

from sklearn.model_selection import train_test_split 

# X is feature matrix (independent variables) usually 2D array like DataFrame or NumPy array - where each row is a sample and each column is a feature
X = data.drop('Exited', axis = 1)   

# y is target array (dependent variable). It contains the values we're trying to predict (Eg. labels in classification or values in regression)
y = data['Exited'] 

# Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Scale these features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### Tensorflow

In [6]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
import datetime

In [8]:
# Build our ANN model 
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1], )), ## HL1 connected with input layer 
    Dense(32, activation = 'relu'), ## HL2 
    Dense(1, activation = 'sigmoid') ## Output layer 
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
import tensorflow 
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [37]:
# Compile the model 
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Set up the TensorBoard 
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [32]:
# Set up Early Stopping 
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [33]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 10s 29ms/step - loss: 0.3307 - accuracy: 0.8651 - val_loss: 0.3550 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3285 - accuracy: 0.8633 - val_loss: 0.3427 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3269 - accuracy: 0.8650 - val_loss: 0.3569 - val_accuracy: 0.8510
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3245 - accuracy: 0.8679 - val_loss: 0.3459 - val_accuracy: 0.8495
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3244 - accuracy: 0.8669 - val_loss: 0.3496 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3211 - accuracy: 0.8689 - val_loss: 0.3625 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3187 - accuracy: 0.8696 - val_loss: 0.3491 - val_accuracy: 0.85

In [34]:
model.save('model.h5')

d:\Code\Python\ann-classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
# Load Tensorboard extension 
# %load_ext tensorboard
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11964), started 0:00:33 ago. (Use '!kill 11964' to kill it.)